In [10]:
import torch
import torchvision.models as models

# Use an existing model from Torchvision, note it 
# will download this if not already on your computer (might take time)
#model = models.alexnet(pretrained=True)

# Create some sample input in the shape this model expects
#dummy_input = torch.randn(10, 3, 224, 224)

# It's optional to label the input and output layers
#input_names = [ "actual_input_1" ] + [ "learned_%d" % i for i in range(16) ]
#output_names = [ "output1" ]

# Use the exporter from torch to convert to onnx 
# model (that has the weights and net arch)
#torch.onnx.export(model, dummy_input, "alexnet.onnx", verbose=True, input_names=input_names, output_names=output_names)

/opt/conda/envs/tensorflow/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/envs/tensorflow/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /home/User/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [02:11<00:00, 1.86MB/s] 


Exported graph: graph(%actual_input_1 : Float(10, 3, 224, 224, strides=[150528, 50176, 224, 1], requires_grad=0, device=cpu),
      %learned_0 : Float(64, 3, 11, 11, strides=[363, 121, 11, 1], requires_grad=1, device=cpu),
      %learned_1 : Float(64, strides=[1], requires_grad=1, device=cpu),
      %learned_2 : Float(192, 64, 5, 5, strides=[1600, 25, 5, 1], requires_grad=1, device=cpu),
      %learned_3 : Float(192, strides=[1], requires_grad=1, device=cpu),
      %learned_4 : Float(384, 192, 3, 3, strides=[1728, 9, 3, 1], requires_grad=1, device=cpu),
      %learned_5 : Float(384, strides=[1], requires_grad=1, device=cpu),
      %learned_6 : Float(256, 384, 3, 3, strides=[3456, 9, 3, 1], requires_grad=1, device=cpu),
      %learned_7 : Float(256, strides=[1], requires_grad=1, device=cpu),
      %learned_8 : Float(256, 256, 3, 3, strides=[2304, 9, 3, 1], requires_grad=1, device=cpu),
      %learned_9 : Float(256, strides=[1], requires_grad=1, device=cpu),
      %learned_10 : Float(409

In [11]:
#HW2-3-1
import onnx
from onnx import helper
alexnet_model = onnx.load('alexnet.onnx')
onnx.checker.check_model(alexnet_model)
print(onnx.helper.printable_graph(alexnet_model.graph))

graph main_graph (
  %actual_input_1[FLOAT, 10x3x224x224]
) initializers (
  %learned_0[FLOAT, 64x3x11x11]
  %learned_1[FLOAT, 64]
  %learned_2[FLOAT, 192x64x5x5]
  %learned_3[FLOAT, 192]
  %learned_4[FLOAT, 384x192x3x3]
  %learned_5[FLOAT, 384]
  %learned_6[FLOAT, 256x384x3x3]
  %learned_7[FLOAT, 256]
  %learned_8[FLOAT, 256x256x3x3]
  %learned_9[FLOAT, 256]
  %learned_10[FLOAT, 4096x9216]
  %learned_11[FLOAT, 4096]
  %learned_12[FLOAT, 4096x4096]
  %learned_13[FLOAT, 4096]
  %learned_14[FLOAT, 1000x4096]
  %learned_15[FLOAT, 1000]
) {
  %/features/features.0/Conv_output_0 = Conv[dilations = [1, 1], group = 1, kernel_shape = [11, 11], pads = [2, 2, 2, 2], strides = [4, 4]](%actual_input_1, %learned_0, %learned_1)
  %/features/features.1/Relu_output_0 = Relu(%/features/features.0/Conv_output_0)
  %/features/features.2/MaxPool_output_0 = MaxPool[ceil_mode = 0, dilations = [1, 1], kernel_shape = [3, 3], pads = [0, 0, 0, 0], strides = [2, 2]](%/features/features.1/Relu_output_0)
  %/featu

In [12]:
for node in alexnet_model.graph.node:
    print(node)

input: "actual_input_1"
input: "learned_0"
input: "learned_1"
output: "/features/features.0/Conv_output_0"
name: "/features/features.0/Conv"
op_type: "Conv"
attribute {
  name: "dilations"
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "group"
  i: 1
  type: INT
}
attribute {
  name: "kernel_shape"
  ints: 11
  ints: 11
  type: INTS
}
attribute {
  name: "pads"
  ints: 2
  ints: 2
  ints: 2
  ints: 2
  type: INTS
}
attribute {
  name: "strides"
  ints: 4
  ints: 4
  type: INTS
}
doc_string: "/opt/conda/envs/tensorflow/lib/python3.9/site-packages/torch/nn/modules/conv.py(456): _conv_forward\n/opt/conda/envs/tensorflow/lib/python3.9/site-packages/torch/nn/modules/conv.py(460): forward\n/opt/conda/envs/tensorflow/lib/python3.9/site-packages/torch/nn/modules/module.py(1501): _slow_forward\n/opt/conda/envs/tensorflow/lib/python3.9/site-packages/torch/nn/modules/module.py(1520): _call_impl\n/opt/conda/envs/tensorflow/lib/python3.9/site-packages/torch/nn/modules/module.py(1511): _wrap

In [4]:
import onnx
from onnx import helper
from onnx import TensorProto

def create_model():
    # 使用ONNX helper functions定义两个输入张量'a'和'b'以及一个权重张量'w'
    a = helper.make_tensor_value_info('a', TensorProto.FLOAT, [1,3])
    b = helper.make_tensor_value_info('b', TensorProto.FLOAT, [1,3])
    w1 = helper.make_tensor_value_info('w1', TensorProto.FLOAT, [3,3])
    w2 = helper.make_tensor_value_info('w2', TensorProto.FLOAT, [3,2])

    # 为了使事情简单，我们来创建一些初始权重数据
    # 但是在实际的模型中，应该在训练过程中来更新这些权重值
    weights1 = helper.make_tensor('w1', TensorProto.FLOAT, [3,3], [1, 2, 3, 4, 5, 6, 7, 8, 9])
    weights2 = helper.make_tensor('w2', TensorProto.FLOAT, [3,2], [1, 2, 3, 4, 5, 6])

    fc1 = helper.make_node(
        'Gemm',
        inputs=['a', 'w1', 'b'],
        outputs=['h1'],
        alpha=1.0,
        beta=1.0,
        transB=1
    )

    fc2 = helper.make_node(
        'Gemm',
        inputs=['h1', 'w2', 'b'],
        outputs=['y'],
        alpha=1.0,
        beta=1.0,
        transB=1
    )

    graph = helper.make_graph(
        [fc1, fc2],
        'TwoLayerFC',
        [a, b, w1, w2],
        [helper.make_tensor_value_info('y', TensorProto.FLOAT, [1,2])],
        value_info=[helper.make_tensor_value_info('h1', TensorProto.FLOAT, [1,3])],
        initializer=[weights1, weights2]
    )

    model = helper.make_model(graph, producer_name='onnx-example')
    return model

model = create_model()

# 检查模型并打印
onnx.checker.check_model(model)
print(onnx.helper.printable_graph(model.graph))
onnx.save(model, 'hw2-3-1-test.onnx')
                        
#https://blog.csdn.net/qq_42178122/article/details/127604762

graph TwoLayerFC (
  %a[FLOAT, 1x3]
  %b[FLOAT, 1x3]
) optional inputs with matching initializers (
  %w1[FLOAT, 3x3]
  %w2[FLOAT, 3x2]
) {
  %h1 = Gemm[alpha = 1, beta = 1, transB = 1](%a, %w1, %b)
  %y = Gemm[alpha = 1, beta = 1, transB = 1](%h1, %w2, %b)
  return %y
}


In [34]:
#hw2-3-1
import onnx
from onnx import helper
from onnx import TensorProto
import numpy as np

def create_model():
    a = helper.make_tensor_value_info('a', TensorProto.FLOAT, [128,128])
    b = helper.make_tensor_value_info('b', TensorProto.FLOAT, [128,128])
    bias = helper.make_tensor_value_info('bias', TensorProto.FLOAT, [128,128])
    
    #weights1 = helper.make_tensor('w1', TensorProto.FLOAT, [3,3], [1, 2, 3, 4, 5, 6, 7, 8, 9])
    # random_data1 = np.random.rand(128, 128).astype(np.float32)
    # weights1 = helper.make_tensor(
    #     name='w1',  # 张量的名称
    #     data_type=TensorProto.FLOAT,  # 数据类型为 FLOAT
    #     dims=[128,128],  # 张量的形状
    #     vals=random_data1.flatten().tolist()  # 将随机数展平并转换为列表
    # )
    fc1 = helper.make_node(
        'Gemm',
        inputs=['a', 'b','bias'],
        outputs=['c'],
        alpha=1.0,
        beta=1.0,
        transB=1
    )


    graph = helper.make_graph(
        [fc1],
        'FC',
        [a, b,bias],
        [helper.make_tensor_value_info('c', TensorProto.FLOAT, [128,128])],
        #value_info=[helper.make_tensor_value_info('h1', TensorProto.FLOAT, [1,3])],
        #initializer=[weights1]
    )

    model = helper.make_model(graph, producer_name='hw2-3-1')
    return model

model = create_model()
onnx.save(model, 'hw2-3-1.onnx')
check_model_ = onnx.load("hw2-3-1.onnx",load_external_data = False)
onnx.checker.check_model(check_model_)

In [14]:
#hw2-3-2
import onnx
from onnx import helper
from onnx import TensorProto
import numpy as np

def create_model():
    a = helper.make_tensor_value_info('a', TensorProto.FLOAT, [128,128])
    b = helper.make_tensor_value_info('b', TensorProto.FLOAT, [128,128])
    bias = helper.make_tensor_value_info('bias', TensorProto.FLOAT, [128,128])
    transpose_node = helper.make_node(
        'Transpose', 
        inputs=['b'], 
        outputs=['bT'], 
        perm=[1, 0]
    )
    split_node_a_0 = helper.make_node(
        'Split',
        inputs=['a'],
        outputs=['a0', 'a1'],
        axis=0,
        num_outputs = 2
    )
    split_node_b_0 = helper.make_node(
        'Split',
        inputs=['bT'],
        outputs=['b0', 'b1'],
        axis=0, 
        num_outputs = 2,
    )
    split_node_a_1_0 = helper.make_node(
        'Split',
        inputs=['a0'],
        outputs=['a00', 'a01'],
        axis=1,
        num_outputs = 2,
    )
    split_node_a_1_1 = helper.make_node(
        'Split',
        inputs=['a1'],
        outputs=['a10','a11'],
        axis=1,
        num_outputs = 2,
    )
    split_node_b_1_0 = helper.make_node(
        'Split',
        inputs=['b0'],
        outputs=['b00', 'b01'],
        axis=1, 
        num_outputs = 2,
    )
    split_node_b_1_1 = helper.make_node(
        'Split',
        inputs=['b1'],
        outputs=['b10','b11'],
        axis=1, 
        num_outputs = 2,
    )
    
    matmul_node_c00_0 = helper.make_node(
        'MatMul',
        inputs=['a00', 'b00'],
        outputs=['c00_0']
    )
    matmul_node_c00_1 = helper.make_node(
        'MatMul',
        inputs=['a01', 'b10'],
        outputs=['c00_1']
    )
    matmul_node_c01_0 = helper.make_node(
        'MatMul',
        inputs=['a00', 'b01'],
        outputs=['c01_0']
    )
    matmul_node_c01_1 = helper.make_node(
        'MatMul',
        inputs=['a01', 'b11'],
        outputs=['c01_1']
    )
    matmul_node_c10_0 = helper.make_node(
        'MatMul',
        inputs=['a10', 'b00'],
        outputs=['c10_0']
    )
    matmul_node_c10_1 = helper.make_node(
        'MatMul',
        inputs=['a11', 'b10'],
        outputs=['c10_1']
    )
    matmul_node_c11_0 = helper.make_node(
        'MatMul',
        inputs=['a10', 'b01'],
        outputs=['c11_0']
    )
    matmul_node_c11_1 = helper.make_node(
        'MatMul',
        inputs=['a11', 'b11'],
        outputs=['c11_1']
    )
    add_node00 = helper.make_node(
        'Add',
        inputs=['c00_0', 'c00_1'],
        outputs=['c00']
    )
    add_node01 = helper.make_node(
        'Add',
        inputs=['c01_0', 'c01_1'],
        outputs=['c01']
    )
    add_node10 = helper.make_node(
        'Add',
        inputs=['c10_0', 'c10_1'],
        outputs=['c10']
    )
    add_node11 = helper.make_node(
        'Add',  # 操作类型为 Add
        inputs=['c11_0', 'c11_1'],
        outputs=['c11']
    )
    concat_node0 = helper.make_node(
        'Concat',
        inputs=['c00', 'c10'],
        outputs=['c0'],
        axis=0
    )
    concat_node1 = helper.make_node(
        'Concat',
        inputs=['c01', 'c11'],
        outputs=['c1'] ,
        axis=0
    )
    concat_node = helper.make_node(
        'Concat',
        inputs=['c0', 'c1'],
        outputs=['c_'],
        axis=1
    )
    bias_node = helper.make_node(
        'Add',
        inputs=['c_', 'bias'],
        outputs=['c'],
    )
    
    graph = helper.make_graph(
        [transpose_node, split_node_a_0, split_node_b_0, split_node_a_1_0,split_node_a_1_1, split_node_b_1_0,split_node_b_1_1,
         matmul_node_c00_0,matmul_node_c00_1,matmul_node_c01_0,matmul_node_c01_1,
         matmul_node_c10_0,matmul_node_c10_1,matmul_node_c11_0,matmul_node_c11_1,
         add_node00, add_node01, add_node10, add_node11,
         concat_node0,concat_node1,concat_node, bias_node],
        'hw2-3-2_Graph',
        [a, b, bias],
        [helper.make_tensor_value_info('c', onnx.TensorProto.FLOAT, [128, 128])]
    )
    model = helper.make_model(graph, producer_name='hw2-3-2')
    return model

# 保存Model

print("ONNX 版本:", onnx.__version__)
model = create_model()
onnx.save(model, 'hw2-3-2.onnx')
check_model_ = onnx.load("hw2-3-2.onnx",load_external_data = False)
onnx.checker.check_model(check_model_)


ONNX 版本: 1.13.1


In [7]:
## hw2-3-3
def get_split_size(length):
    num = length //2048
    remainder = length % 2048
    split_size = []
    for i in range(num):
        split_size.append(2048)#2048
    if(remainder != 0):
        num += 1
        split_size.append(remainder)
    return split_size, num
        
    
def build_new_nodes(idx, my_modified_alexnet, M, N, K, input0, input1, input2, output0):
    transpose_node = helper.make_node(
        'Transpose', 
        inputs=[input1], 
        outputs=[str(idx)+'bT'], 
        perm=[1, 0]
    )
    my_modified_alexnet.node.extend([transpose_node])
    split_sizeN, numN = get_split_size(N)
    output_list_a = []
    for i in range(len(split_sizeN)):
        output_list_a.append(str(idx)+'a'+str(i))
    split_node_a = helper.make_node(
        'Split',
        inputs=[input0],
        outputs=output_list_a,
        axis=1,
        #split=split_sizeN,
        num_outputs = numN
    )
    my_modified_alexnet.node.extend([split_node_a])
    output_list_b = []
    for i in range(len(split_sizeN)):
        output_list_b.append(str(idx)+'b'+str(i))
    split_node_b_0 = helper.make_node(
        'Split',
        inputs=[str(idx)+'bT'],
        outputs=output_list_b,
        axis=0,
        #split=split_sizeN,
        num_outputs = numN
    )
    my_modified_alexnet.node.extend([split_node_b_0])
    split_sizeK, numK = get_split_size(K)
    for i in output_list_b:
        output_list_b1 = []
        for j in range(len(split_sizeK)):
            output_list_b1.append(i+'_'+str(j))
        #print(output_list_b1)
        split_node_b_1 = helper.make_node(
            'Split',
            inputs=[i],
            outputs=output_list_b1,
            axis=1,
            #split=split_sizeK,
            num_outputs = numK
        )
        my_modified_alexnet.node.extend([split_node_b_1])

    for i in range(numK):
        zero_data = np.zeros((1, split_sizeK[i]), dtype=np.float32)
        c_name = str(idx)+'c'+str(i)
        zero_tensor = helper.make_tensor(
            name=c_name,
            data_type=TensorProto.FLOAT,
            dims=[1, split_sizeK[i]],
            vals=zero_data.flatten().tolist()
        )
        add_input_names_ = []
        for j in range(numN):
            inputs_a = str(idx)+'a'+str(j)
            inputs_b = str(idx)+'b'+str(j)+'_'+str(i)
            output_c = str(idx)+'c'+str(i)+'_'+str(j)
            
            print(inputs_a,inputs_b)
            multi_node = helper.make_node(
                'MatMul',
                inputs=[inputs_a, inputs_b],
                outputs=[output_c]
            )
            add_input_names_.append(output_c)
            my_modified_alexnet.node.extend([multi_node])

        id_1st_node = helper.make_node(
            'Identity', 
            inputs=[str(idx)+'c'+str(i)+'_0'], 
            outputs=[str(idx)+str(i)+'add_result0']
        )
        my_modified_alexnet.node.extend([id_1st_node])
        for k in range(numN-1):
            in0 = str(idx)+str(i)+'add_result'+str(k)
            in1 = str(idx)+'c'+str(i)+'_'+str(k+1)
            add_node = helper.make_node(
                'Add',
                inputs=[in0,in1],
                outputs=[str(idx)+str(i)+'add_result' + str(k+1)],
            )
            my_modified_alexnet.node.extend([add_node])
        final_add_node = helper.make_node(
            'Identity', 
            inputs=[str(idx)+str(i)+'add_result'+str(numN-1)], 
            outputs=[c_name]
        )
        my_modified_alexnet.node.extend([final_add_node])

    output_c = []
    for c in range(numK):
        tmp = str(idx)+'c' + str(c)
        output_c.append(tmp)
    concat_node = helper.make_node(
        'Concat',
        inputs = output_c,
        outputs = [str(idx)+'c_'],
        axis=1
    )
    my_modified_alexnet.node.extend([concat_node])
    add_node_bias = helper.make_node(
        'Add', 
        inputs=[str(idx)+'c_', input2],
        outputs=output0,
    )
    my_modified_alexnet.node.extend([add_node_bias])
            
            

In [8]:
import onnx
from onnx import helper
from onnx import TensorProto
import numpy as np

# Load the AlexNet ONNX model
alexnet_model = onnx.load('alexnet.onnx')
graph_proto = alexnet_model.graph




input_info = graph_proto.input


output_info = graph_proto.output
tile_size = 64
layer_name = []
input_name = []
output_name = []
for node in graph_proto.node:
    print(node.name)
    if('Gemm' in node.name):
        print("haha!")
        layer_name.append(node.name)
        input_name.append(node.input)
        output_name.append(node.output)
    #print(output_name[-1])
    print("====================")



my_modified_alexnet = onnx.helper.make_graph(
    nodes = [],
    name = "my_modified_alexnet",
    inputs = graph_proto.input,
    outputs = graph_proto.output,
    initializer = graph_proto.initializer
)

for node in graph_proto.node:
    if(node.name == layer_name[0]):
        input0,input1,input2 = input_name[0]
        output0 = output_name[0]
        M = 1
        N = 9216
        K = 4096
        build_new_nodes(0, my_modified_alexnet, M, N, K,input0,input1,input2,output0)
    
    elif(node.name == layer_name[1]):
        input0,input1,input2 = input_name[1]
        output0 = output_name[1]
        M = 1
        N = 4096
        K = 4096
        build_new_nodes(1, my_modified_alexnet, M, N, K,input0,input1,input2,output0)
    elif(node.name == layer_name[2]):
        input0,input1,input2 = input_name[2]
        output0 = output_name[2]
        M = 1
        N = 4096
        K = 1000
        build_new_nodes(2, my_modified_alexnet, M, N, K,input0,input1,input2,output0)
    else:
        my_modified_alexnet.node.extend([node])
        

model = helper.make_model(my_modified_alexnet, producer_name='my_modified_alexnet')

onnx.save(model, 'my_modified_alexnet.onnx')




/features/features.0/Conv
/features/features.1/Relu
/features/features.2/MaxPool
/features/features.3/Conv
/features/features.4/Relu
/features/features.5/MaxPool
/features/features.6/Conv
/features/features.7/Relu
/features/features.8/Conv
/features/features.9/Relu
/features/features.10/Conv
/features/features.11/Relu
/features/features.12/MaxPool
/avgpool/AveragePool
/Flatten
/classifier/classifier.1/Gemm
haha!
/classifier/classifier.2/Relu
/classifier/classifier.4/Gemm
haha!
/classifier/classifier.5/Relu
/classifier/classifier.6/Gemm
haha!
0a0 0b0_0
0a1 0b1_0
0a2 0b2_0
0a3 0b3_0
0a4 0b4_0
0a0 0b0_1
0a1 0b1_1
0a2 0b2_1
0a3 0b3_1
0a4 0b4_1
1a0 1b0_0
1a1 1b1_0
1a0 1b0_1
1a1 1b1_1
2a0 2b0_0
2a1 2b1_0


In [9]:
import onnxruntime as ort
check_model_ = onnx.load("my_modified_alexnet.onnx",load_external_data = False)
onnx.checker.check_model(check_model_)

my_onnx_session = ort.InferenceSession("my_modified_alexnet.onnx")
onnx_session = ort.InferenceSession("alexnet.onnx")

input_name = onnx_session.get_inputs()[0].name
input = np.random.rand(10,3,224,224).astype(np.float32)

onnx_output = onnx_session.run(None, {input_name: input})
my_onnx_output = my_onnx_session.run(None, {input_name: input})

print(np.allclose(np.array(my_onnx_output), np.array(onnx_output), atol = 1e-6))
print(np.array(my_onnx_output).shape)

True
(1, 10, 1000)
